## 라이브러리

In [1]:
### 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 한글 처리
plt.rc("font", family = "Malgun Gothic")

### 기호 처리
plt.rcParams["axes.unicode_minus"] = False

In [28]:
### 정규화 패키지
from sklearn.preprocessing import StandardScaler

### 모델 관련 패키지
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from os import listdir
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

import platform

---

## 데이터 확인 결과

1. 원본 데이터는 사람이 작성한 발주 수량 예측 데이터임
2. 모델 학습을 통해 수요를 예측하면 오차를 줄일 수 있을 것으로 가정함
3. 시각화를 통해 긴급 발주되는 부품이 있음을 확인함
4. 긴급 발주 부품은 발주 수량 예측이 어려울 것으로 판단하여 일관성 있게 발주되는 부품 중 5개를 추출하여 학습해보기로 결정함
5. 이전 시간 단계의 정보를 기억하고 활용하여 좋은 성능을 보이는 LSTM 모델과, 회귀 모델 중 과적합을 방지하는데 용이한 앙상블 모델을 사용하며, 비교 후 가장 오차가 적은 모델을 선정할 계획

---

## LSTM 모델 함수

### 1. 데이터 불러오기 함수(getData)

In [12]:
def getData(part) :
    ### 데이터 불러오기
    data = pd.read_csv("./data_new/01_전처리후_파트별_데이터/part{}_data.csv".format(part))
    
    return data

### 2. 주요 변수 선택 함수(getCols)

In [5]:
def getCols(data, num) :
    ### 주요 변수 선택 및 저장
    data = data.iloc[:, [1, num+1, num+2, num+3]].reset_index(drop=True)
    
    return data

### 3. 독립/종속변수 함수(to_timeseries_data)

In [7]:
### 독립변수와 종속변수 만드는 함수
def to_timeseries_data(data, lookback, delay):
    
    output_len = len(data)-(lookback+delay)+1 # N=total_length-(3+3)+1
    n_feature = data.shape[-1] # =4
    
    inputs = np.zeros((output_len, lookback, n_feature)) # (N,3,4)
    targets = np.zeros((output_len,)) # (N,)
    
    for i in range(output_len):
        inputs[i] = data.iloc[i:i+lookback, :]
        targets[i] = data.iloc[i+lookback+delay-1, 0]
        
    return inputs, targets

### 4. 데이터 분리 및 정규화 함수(getSplitData)

In [18]:
def getSplitData(data, num) :
    ### 사용자 정의 함수 적용
    X_part, y_part = to_timeseries_data(data, 3, num)

    print("X의 형태: ", X_part.shape)
    print("y의 형태: ", y_part.shape)
    
    ### 데이터셋 분리
    X_train_part, X_val_part, X_test_part = np.split(X_part, [int(0.7*len(X_part)), int(0.8*len(X_part))])
    y_train_part, y_val_part, y_test_part = np.split(y_part, [int(0.7*len(y_part)), int(0.8*len(y_part))])

    ### 분리 이후 데이터 형태
    print("X 학습: {}, X 검증: {}, X 평가: {}".format(X_train_part.shape,X_val_part.shape,X_test_part.shape))
    print("y 학습: {}, y 검증: {}, y 평가: {}".format(y_train_part.shape,y_val_part.shape,y_test_part.shape))
    
    ### 데이터 정규화
    Xscaler_part = StandardScaler()
    X_train_part = Xscaler_part.fit_transform(X_train_part.reshape(-1, X_train_part.shape[-1])).reshape(X_train_part.shape)
    X_val_part = Xscaler_part.transform(X_val_part.reshape(-1, X_val_part.shape[-1])).reshape(X_val_part.shape)
    X_test_part = Xscaler_part.transform(X_test_part.reshape(-1, X_test_part.shape[-1])).reshape(X_test_part.shape)
    X_list = [Xscaler_part, X_train_part, X_val_part, X_test_part]
    
    yscaler_part = StandardScaler()
    y_train_part = yscaler_part.fit_transform(y_train_part.reshape(-1,1))
    y_val_part = yscaler_part.transform(y_val_part.reshape(-1,1))
    y_test_part = yscaler_part.transform(y_test_part.reshape(-1,1))
    y_list = [yscaler_part, y_train_part, y_val_part, y_test_part]
    
    return X_list, y_list

### 5. LSTM 모델 함수(getLSTMModel)

In [130]:
def getLSTMModel(part, num, X_list, y_list) :
    ### 신경망 모델 생성하기
    model = Sequential()
    
    ### LSTM 계층 추가하기
    model.add(LSTM(8, dropout=0.2, activation='leaky_relu', input_shape=(3,4), return_sequences=True))
    model.add(LSTM(8, dropout=0.2, activation='leaky_relu'))
    
    ### 출력 계층 추가하기
    model.add(Dense(1, activation='linear'))

    ### 모델 상태 확인하기
    model.summary()
    
    ### 모델 설정하기
    model.compile(optimizer='adam', loss='mse')
    
    ### 모델 저장 경로 지정
    model_path = './models/part{}_d{}_lstm.h5'.format(part, num)

    ### 콜백 함수 지정
    callbacks = [EarlyStopping(monitor='val_loss', patience=15),
                ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=0, save_best_only=True)]
    
    ### 모델 훈련시키기
    history = model.fit(X_list[1], y_list[1], epochs=100, batch_size=4, validation_data=(X_list[2], y_list[2]),
                        callbacks=callbacks)
    
    ### 모델 불러오기
    best_model = load_model('./models/part{}_d{}_lstm.h5'.format(part, num))
    
    ### 테스트 데이터로 성능 평가
    best_model.evaluate(X_list[3], y_list[3])
    
    ### 테스트 데이터로 예측하기
    y_pred_part = best_model.predict(X_list[3])
    
    ### 예측값을 기존 값 범위로 역변환
    y_pred_part_inv = y_list[0].inverse_transform(y_pred_part)
    y_test_part_inv = y_list[0].inverse_transform(y_list[3])
    
    ### mae, mse, rmse 계산
    mae = mean_absolute_error(y_test_part_inv, y_pred_part_inv)
    mse = mean_squared_error(y_test_part_inv, y_pred_part_inv)
    rmse = np.sqrt(mse)

    ### 모델 성능 계산
    train_eva = best_model.evaluate(X_list[1], y_list[1])
    val_eva = best_model.evaluate(X_list[2], y_list[2])
    test_eva = best_model.evaluate(X_list[3], y_list[3])
    
    result_list = [part, num, train_eva, val_eva, test_eva, mae, mse, rmse]
    
    return result_list, y_test_part_inv, y_pred_part_inv

In [50]:
df = pd.DataFrame(columns=['part_number', 'pred_day','train_evaluate', 'val_evaluate', 'test_evaluate', 'mae', 'mse', 'rmse'])

for part in [6, 15, 16, 29, 94]:
    data = getData(part)
    
    for num in range(1, 8, 1) :
        data_new = getCols(data, num)
        
        X_list, y_list = getSplitData(data_new, num)
        result_list, y_test_part_inv, y_pred_part_inv = getLstmModel(part, num, X_list, y_list)
        
        df.loc[len(df)] = result_list
df.to_csv('./data_new/lstm_결과.csv', index=False, encoding='utf-8-sig')

X의 형태:  (46, 3, 4)
y의 형태:  (46,)
X 학습: (32, 3, 4), X 검증: (4, 3, 4), X 평가: (10, 3, 4)
y 학습: (32,), y 검증: (4,), y 평가: (10,)
Model: "sequential_144"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_288 (LSTM)             (None, 3, 8)              416       
                                                                 
 lstm_289 (LSTM)             (None, 8)                 544       
                                                                 
 dense_144 (Dense)           (None, 1)                 9         
                                                                 
Total params: 969
Trainable params: 969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8/8 [==============================] - 3s 62ms/step - loss: 0.9783 - val_loss: 0.1620
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 0.9791 - val_loss: 0.1626


### 6. 랜덤포레스트 모델 함수(getRFModel)

In [75]:
def getRFModel(part, num, X_list, y_list):
    ### 랜덤 포레스트 모델 생성하기
    model = RandomForestRegressor(n_estimators=10, random_state=42)
    
    ### 2차원 배열로 변경
    X_train = X_list[1].reshape(X_list[1].shape[0], -1)
    X_val = X_list[2].reshape(X_list[2].shape[0], -1)
    X_test = X_list[3].reshape(X_list[3].shape[0], -1)

    ### 모델 훈련시키기
    model.fit(X_train, y_list[1])
    
    ### 테스트 데이터로 성능 평가  
    y_pred_part = model.predict(X_test)
    
    ### 예측값을 기존 값 범위로 역변환
    y_pred_part_inv = y_list[0].inverse_transform(y_pred_part.reshape(-1, 1))
    y_test_part_inv = y_list[0].inverse_transform(y_list[3])
    
    ### mae, mse, rmse 계산
    mae = mean_absolute_error(y_test_part_inv, y_pred_part_inv)
    mse = mean_squared_error(y_test_part_inv, y_pred_part_inv)
    rmse = np.sqrt(mse)
    
    ### 모델 성능 계산
    train_eva = model.score(X_train, y_list[1])
    val_eva = model.score(X_val, y_list[2])
    test_eva = model.score(X_test, y_list[3])
    
    result_list = [part, num, train_eva, val_eva, test_eva, mae, mse, rmse]
    
    return result_list, y_test_part_inv, y_pred_part_inv

### 7. XGBoost 모델 함수(getXGBModel)

In [73]:
def getXGBModel(part, num, X_list, y_list):
    ### XGBoost 모델 생성하기
    model = xgb.XGBRegressor(n_estimators=10, random_state=42)
    
    ### 2차원 배열로 변경
    X_train = X_list[1].reshape(X_list[1].shape[0], -1)
    X_val = X_list[2].reshape(X_list[2].shape[0], -1)
    X_test = X_list[3].reshape(X_list[3].shape[0], -1)

    ### 모델 훈련시키기
    model.fit(X_train, y_list[1])
    
    ### 테스트 데이터로 성능 평가  
    y_pred_part = model.predict(X_test)
    
    ### 예측값을 기존 값 범위로 역변환
    y_pred_part_inv = y_list[0].inverse_transform(y_pred_part.reshape(-1, 1))
    y_test_part_inv = y_list[0].inverse_transform(y_list[3])
    
    ### mae, mse, rmse 계산
    mae = mean_absolute_error(y_test_part_inv, y_pred_part_inv)
    mse = mean_squared_error(y_test_part_inv, y_pred_part_inv)
    rmse = np.sqrt(mse)
    
    ### 모델 성능 계산
    train_eva = model.score(X_train, y_list[1])
    val_eva = model.score(X_val, y_list[2])
    test_eva = model.score(X_test, y_list[3])
    
    result_list = [part, num, train_eva, val_eva, test_eva, mae, mse, rmse]
    
    return result_list, y_test_part_inv, y_pred_part_inv

### 8. GradientBoosting 모델 함수(getGBModel)

In [72]:
def getGBModel(part, num, X_list, y_list):
    ### Gradient Boosting 모델 생성하기
    model = GradientBoostingRegressor(n_estimators=10, random_state=42)
    
    ### 2차원 배열로 변경
    X_train = X_list[1].reshape(X_list[1].shape[0], -1)
    X_val = X_list[2].reshape(X_list[2].shape[0], -1)
    X_test = X_list[3].reshape(X_list[3].shape[0], -1)

    ### 모델 훈련시키기
    model.fit(X_train, y_list[1])
    
    ### 테스트 데이터로 성능 평가  
    y_pred_part = model.predict(X_test)
    
    ### 예측값을 기존 값 범위로 역변환
    y_pred_part_inv = y_list[0].inverse_transform(y_pred_part.reshape(-1, 1))
    y_test_part_inv = y_list[0].inverse_transform(y_list[3])
    
    ### mae, mse, rmse 계산
    mae = mean_absolute_error(y_test_part_inv, y_pred_part_inv)
    mse = mean_squared_error(y_test_part_inv, y_pred_part_inv)
    rmse = np.sqrt(mse)
    
    ### 모델 성능 계산
    train_eva = model.score(X_train, y_list[1])
    val_eva = model.score(X_val, y_list[2])
    test_eva = model.score(X_test, y_list[3])
    
    result_list = [part, num, train_eva, val_eva, test_eva, mae, mse, rmse]
    
    return result_list, y_test_part_inv, y_pred_part_inv

### 9. HistGradientBoosting 모델 함수(getHGBModel)

In [71]:
def getHGBModel(part, num, X_list, y_list):
    ### Histogram Gradient Boosting 모델 생성하기
    model = HistGradientBoostingRegressor(max_iter=10, random_state=42)
    
    ### 2차원 배열로 변경
    X_train = X_list[1].reshape(X_list[1].shape[0], -1)
    X_val = X_list[2].reshape(X_list[2].shape[0], -1)
    X_test = X_list[3].reshape(X_list[3].shape[0], -1)

    ### 모델 훈련시키기
    model.fit(X_train, y_list[1])
    
    ### 테스트 데이터로 성능 평가  
    y_pred_part = model.predict(X_test)
    
    ### 예측값을 기존 값 범위로 역변환
    y_pred_part_inv = y_list[0].inverse_transform(y_pred_part.reshape(-1, 1))
    y_test_part_inv = y_list[0].inverse_transform(y_list[3])
    
    ### mae, mse, rmse 계산
    mae = mean_absolute_error(y_test_part_inv, y_pred_part_inv)
    mse = mean_squared_error(y_test_part_inv, y_pred_part_inv)
    rmse = np.sqrt(mse)
    
    ### 모델 성능 계산
    train_eva = model.score(X_train, y_list[1])
    val_eva = model.score(X_val, y_list[2])
    test_eva = model.score(X_test, y_list[3])
    
    result_list = [part, num, train_eva, val_eva, test_eva, mae, mse, rmse]
    
    return result_list, y_test_part_inv, y_pred_part_inv

### 함수 실행하기

In [76]:
# 각 모델의 결과를 저장할 데이터프레임 생성
models = ['LSTM', 'RF', 'XGB', 'GB', 'HGB']
dfs = {model: pd.DataFrame(columns=['part_number', 'pred_day', 'train_evaluate', 'val_evaluate', 'test_evaluate', 'mae', 'mse', 'rmse']) for model in models}

# 분석할 부품(part) 리스트
parts = [6, 15, 16, 29, 94]

# 각 부품에 대해 반복
for part in parts:
    data = getData(part)
    
    # 1부터 7까지의 범위로 반복
    for num in range(1, 8, 1):
        data_new = getCols(data, num)
        
        X_list, y_list = getSplitData(data_new, num)
        
        # 각 모델에 대한 결과를 얻어서 해당 데이터프레임에 추가
        for model in models:
            result_list, _, _ = globals()[f'get{model}Model'](part, num, X_list, y_list)
            dfs[model].loc[len(dfs[model])] = result_list

# 결과를 각 모델에 대한 CSV 파일로 저장
for model in models:
    dfs[model].to_csv(f'./data_new/{model.lower()}_결과.csv', index=False, encoding='utf-8-sig')

X의 형태:  (46, 3, 4)
y의 형태:  (46,)
X 학습: (32, 3, 4), X 검증: (4, 3, 4), X 평가: (10, 3, 4)
y 학습: (32,), y 검증: (4,), y 평가: (10,)
Model: "sequential_217"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_434 (LSTM)             (None, 3, 8)              416       
                                                                 
 lstm_435 (LSTM)             (None, 8)                 544       
                                                                 
 dense_217 (Dense)           (None, 1)                 9         
                                                                 
Total params: 969
Trainable params: 969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8/8 [==============================] - 3s 66ms/step - loss: 1.0036 - val_loss: 0.1349
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 0.9982 - val_loss: 0.1357
E

C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X의 형태:  (45, 3, 4)
y의 형태:  (45,)
X 학습: (31, 3, 4), X 검증: (5, 3, 4), X 평가: (9, 3, 4)
y 학습: (31,), y 검증: (5,), y 평가: (9,)
Model: "sequential_218"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_436 (LSTM)             (None, 3, 8)              416       
                                                                 
 lstm_437 (LSTM)             (None, 8)                 544       
                                                                 
 dense_218 (Dense)           (None, 1)                 9         
                                                                 
Total params: 969
Trainable params: 969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8/8 [==============================] - 4s 75ms/step - loss: 1.0002 - val_loss: 0.0542
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 1.0035 - val_loss: 0.0574
Ep

C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model: "sequential_219"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_438 (LSTM)             (None, 3, 8)              416       
                                                                 
 lstm_439 (LSTM)             (None, 8)                 544       
                                                                 
 dense_219 (Dense)           (None, 1)                 9         
                                                                 
Total params: 969
Trainable params: 969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8/8 [==============================] - 4s 37ms/step - loss: 1.0094 - val_loss: 0.0677
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 1.0031 - val_loss: 0.0690
Epoch 3/100
8/8 [==============================] - 0s 12ms/step - loss: 0.9978 - val_loss: 0.0673
Epoch 4/100
8/8 [========

...

C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X의 형태:  (41, 3, 4)
y의 형태:  (41,)
X 학습: (28, 3, 4), X 검증: (4, 3, 4), X 평가: (9, 3, 4)
y 학습: (28,), y 검증: (4,), y 평가: (9,)
Model: "sequential_250"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_500 (LSTM)             (None, 3, 8)              416       
                                                                 
 lstm_501 (LSTM)             (None, 8)                 544       
                                                                 
 dense_250 (Dense)           (None, 1)                 9         
                                                                 
Total params: 969
Trainable params: 969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 5s 155ms/step - loss: 1.0089 - val_loss: 0.9045
Epoch 2/100
7/7 [==============================] - 0s 15ms/step - loss: 0.9907 - val_loss: 0.9107
E

C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X의 형태:  (40, 3, 4)
y의 형태:  (40,)
X 학습: (28, 3, 4), X 검증: (4, 3, 4), X 평가: (8, 3, 4)
y 학습: (28,), y 검증: (4,), y 평가: (8,)
Model: "sequential_251"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_502 (LSTM)             (None, 3, 8)              416       
                                                                 
 lstm_503 (LSTM)             (None, 8)                 544       
                                                                 
 dense_251 (Dense)           (None, 1)                 9         
                                                                 
Total params: 969
Trainable params: 969
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 5s 137ms/step - loss: 1.0393 - val_loss: 1.1933
Epoch 2/100
7/7 [==============================] - 0s 19ms/step - loss: 1.0463 - val_loss: 1.1823
E

C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\siyoon\anaconda3\envs\project\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 각 모델 결과 비교

In [85]:
# 5개의 모델 리스트
models = ['lstm', 'rf', 'xgb', 'gb', 'hgb']
# 모든 결과를 저장할 데이터프레임
final_rmse_df = pd.DataFrame()

# 각 모델에 대해 작업 수행
for model in models:
    # 모델 결과 데이터 로드
    df = pd.read_csv(f"./data_new/{model}_결과.csv")
    # part_number 기준으로 rmse의 평균을 계산
    result_df = df.groupby('part_number')['rmse'].mean().reset_index()
    # 결과 데이터프레임에 열 방향으로 합치기
    final_rmse_df = pd.concat([final_rmse_df, result_df['rmse']], axis=1, ignore_index=True)

# 각 모델명을 컬럼명으로 설정
final_rmse_df.columns = models

# 저장하기
final_rmse_df.to_csv('./data_new/모델별_rmse_평균.csv', index=False, encoding='utf-8-sig')

# 최종 결과 출력
final_rmse_df

,lstm,rf,xgb,gb,hgb
0,8.874576,12.817189,12.883608,12.270567,9.435244
1,9.446987,10.150671,14.300918,10.865720,9.596393
2,15.963830,18.331679,19.382355,17.576702,16.641397
3,9.741370,17.067289,40.694170,13.159314,15.048942
4,6.102161,7.227756,6.590934,6.758569,6.245571


lstm과 앙상블 모델을 학습 후 rmse를 기준으로 비교함<br>
각 부품 넘버별 D+1~7의 평균 rmse를 비교하기로 함<br><br>

비교 결과, 모든 부품에서 lstm의 rmse 평균이 가장 낮음<br>
따라서, lstm 모델이 가장 적합한 모델이라 판단하여 최종 모델로 선정함<br>

---

## LSTM 모델 튜닝

### LSTM 모델 튜닝 함수(getLSTMTuningModel)

LSTM 모델의 loss가 높게 나와 손실 가중치를 주어 loss를 줄이고자 함. loss_weights=0.01로 주고 실행<br>
그 외의 하이퍼파라미터 튜닝은 아래와 같이 실험을 해봄

1. LSTM 뉴런 수 조정 (8, 16, 32, 64) : 32 선정
2. Dropout 조정 (->0.1~0.5) : 0.2 선정
3. 활성화 함수 변경 (relu, leaky_relu) : leaky_relu 선정
4. 옵티마이저 변경 (adam, sgd) : adam 선정

In [179]:
def getLSTMTuningModel(part, num, X_list, y_list) :
    ### 신경망 모델 생성하기
    model = Sequential()
    
    ### LSTM 계층 추가하기
    model.add(LSTM(32, dropout=0.2, activation='leaky_relu', input_shape=(3, 4), return_sequences=True))
    model.add(LSTM(32, dropout=0.2, activation='leaky_relu'))
    
    ### 출력 계층 추가하기
    model.add(Dense(1, activation='linear'))

    ### 모델 상태 확인하기
    model.summary()
    
    ### 모델 설정하기
    model.compile(optimizer='adam', loss='mse', loss_weights=0.01)
    
    ### 모델 저장 경로 지정
    model_path = './models/part{}_d{}_lstm_tuning.h5'.format(part, num)

    ### 콜백 함수 지정
    callbacks = [EarlyStopping(monitor='val_loss', patience=15),
                ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=0, save_best_only=True)]
    
    ### 모델 훈련시키기
    history = model.fit(X_list[1], y_list[1], epochs=100, batch_size=4, validation_data=(X_list[2], y_list[2]),
                        callbacks=callbacks)
    
    ### 모델 불러오기
    best_model = load_model('./models/part{}_d{}_lstm_tuning.h5'.format(part, num))
    
    ### 테스트 데이터로 성능 평가
    best_model.evaluate(X_list[3], y_list[3])
    
    ### 테스트 데이터로 예측하기
    y_pred_part = best_model.predict(X_list[3])
    
    ### 예측값을 기존 값 범위로 역변환
    y_pred_part_inv = y_list[0].inverse_transform(y_pred_part)
    y_test_part_inv = y_list[0].inverse_transform(y_list[3])
    
    ### mae, mse, rmse 계산
    mae = mean_absolute_error(y_test_part_inv, y_pred_part_inv)
    mse = mean_squared_error(y_test_part_inv, y_pred_part_inv)
    rmse = np.sqrt(mse)

    ### 모델 성능 계산
    train_eva = best_model.evaluate(X_list[1], y_list[1])
    val_eva = best_model.evaluate(X_list[2], y_list[2])
    test_eva = best_model.evaluate(X_list[3], y_list[3])
    
    result_list = [part, num, train_eva, val_eva, test_eva, mae, mse, rmse]
    
    return result_list, y_test_part_inv, y_pred_part_inv

### 함수 실행하기

In [180]:
df = pd.DataFrame(columns=['part_number', 'pred_day','train_evaluate', 'val_evaluate', 'test_evaluate', 'mae', 'mse', 'rmse'])

for part in [6, 15, 16, 29, 94]:
    data = getData(part)
    
    for num in range(1, 8, 1) :
        data_new = getCols(data, num)

        X_list, y_list = getSplitData(data_new, num)
        result_list, y_test_part_inv, y_pred_part_inv = getLSTMTuningModel(part, num, X_list, y_list)
        
        df.loc[len(df)] = result_list
df.to_csv('./data_new/lstm_결과_튜닝후.csv', index=False, encoding='utf-8-sig')

X의 형태:  (46, 3, 4)
y의 형태:  (46,)
X 학습: (32, 3, 4), X 검증: (4, 3, 4), X 평가: (10, 3, 4)
y 학습: (32,), y 검증: (4,), y 평가: (10,)
Model: "sequential_987"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1974 (LSTM)            (None, 3, 32)             4736      
                                                                 
 lstm_1975 (LSTM)            (None, 32)                8320      
                                                                 
 dense_987 (Dense)           (None, 1)                 33        
                                                                 
Total params: 13,089
Trainable params: 13,089
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8/8 [==============================] - 4s 120ms/step - loss: 0.0100 - val_loss: 0.0010
Epoch 2/100
8/8 [==============================] - 0s 9ms/step - loss: 0.0098 - val_loss: 0

### 튜닝 전/후 결과 비교

In [140]:
# 튜닝 전 loss 확인
df_before = pd.read_csv("./data_new/lstm_결과.csv")
df_before

,part_number,pred_day,train_evaluate,val_evaluate,test_evaluate,mae,mse,rmse
0,6.0,1.0,0.994169,0.134280,0.114758,7.446828,99.603179,9.980139
1,6.0,2.0,1.001938,0.054181,0.095073,7.164919,85.178664,9.229229
2,6.0,3.0,0.743079,0.012324,0.065070,6.330399,60.107657,7.752913
3,6.0,4.0,0.820884,0.045073,0.087792,7.460961,80.965769,8.998098
4,6.0,5.0,1.023650,0.038999,0.064452,6.434555,61.326307,7.831111
5,6.0,6.0,1.003540,0.052521,0.080071,6.991891,78.684121,8.870407
6,6.0,7.0,1.000402,0.022159,0.090991,7.730541,89.494174,9.460136
7,15.0,1.0,1.001315,2.303906,0.734188,6.713324,85.755857,9.260446
8,15.0,2.0,0.996900,1.842325,0.793552,7.364521,93.728294,9.681337
9,15.0,3.0,0.984347,1.899140,0.604273,6.738096,73.257395,8.559053


In [181]:
# 튜닝 후 loss 확인
df_after = pd.read_csv("./data_new/lstm_결과_튜닝후.csv")
df_after

,part_number,pred_day,train_evaluate,val_evaluate,test_evaluate,mae,mse,rmse
0,6.0,1.0,0.009852,0.001025,0.001088,7.212905,94.467174,9.719423
1,6.0,2.0,0.009940,0.000651,0.001077,7.497284,96.451575,9.820976
2,6.0,3.0,0.007636,0.000116,0.000853,6.708657,78.769720,8.875231
3,6.0,4.0,0.007412,0.000421,0.000806,7.274001,74.317182,8.620741
4,6.0,5.0,0.009999,0.000653,0.000955,7.307207,90.903285,9.534321
5,6.0,6.0,0.010002,0.000598,0.000878,7.254711,86.233902,9.286221
6,6.0,7.0,0.009877,0.000304,0.001017,8.192920,100.059389,10.002969
7,15.0,1.0,0.009841,0.023447,0.006972,6.494264,81.432742,9.024009
8,15.0,2.0,0.009984,0.017873,0.008253,7.481616,97.479091,9.873150
9,15.0,3.0,0.009497,0.018122,0.006724,6.965995,81.512813,9.028445


손실 가중치를 준 튜닝 후, loss가 줄어들어 모델이 최적화된 것을 확인함

In [184]:
# 모든 결과를 저장할 데이터프레임
final_rmse_df = pd.DataFrame()

# 튜닝 전 rmse 확인
result_before = df_before.groupby('part_number')['rmse'].mean().reset_index()

# 결과 데이터프레임에 열 방향으로 합치기
final_rmse_df = pd.concat([final_rmse_df, result_before['rmse']], axis=1, ignore_index=True)

# 튜닝 후 rmse 확인
result_after = df_after.groupby('part_number')['rmse'].mean().reset_index()

# 결과 데이터프레임에 열 방향으로 합치기
final_rmse_df = pd.concat([final_rmse_df, result_after['rmse']], axis=1, ignore_index=True)

# 컬럼명 설정
final_rmse_df.columns = ['before', 'after']

# 저장하기
final_rmse_df.to_csv('./data_new/lstm_튜닝전_후_rmse_평균.csv', index=False, encoding='utf-8-sig')

# 최종 결과 출력
final_rmse_df

,before,after
0,8.874576,9.408555
1,9.446987,9.684718
2,15.963830,15.692715
3,9.741370,9.603837
4,6.102161,6.089528


### LSTM 튜닝 전/후 RMSE 비교 결과

```
      before      after
0   8.874576   9.408555
1   9.446987   9.684718
2  15.963830  15.692715
3   9.741370   9.603837
4   6.102161   6.089528
```

부품 6과 15의 RMSE는 증가했지만 16, 29, 94의 RMSE는 감소함<br>
튜닝을 통해 loss를 줄일 수 있었으며, 3개의 부품에서 더 나은 결과를 보임

---

→ 03_원본데이터_오차계산 파일에서 최종 검증 진행